### Reference
https://github.com/huggingface/diffusers/blob/main/examples/dreambooth/README_sdxl.md

### Check CUDA

In [ ]:
import subprocess
# GPU Check

# @markdown The system checks for a compatible GPU with enough memory and installs necessary Python packages during setup.
# Run the nvidia-smi command to get the VRAM information
result = subprocess.run(["nvidia-smi", "--query-gpu=name,memory.total,memory.free",
                        "--format=csv,noheader"], capture_output=True, check=True)

# Split the output by newline characters to get a list of VRAM info for each GPU
vram_info = result.stdout.decode("utf-8").strip().split("\n")

# Parse the VRAM info for each GPU
for info in vram_info:
    name, total, free = info.split(",")
    total = int(total.strip().split()[0])  # Total VRAM in MB
    free = int(free.strip().split()[0])  # Free VRAM in MB

    print(f"GPU: {name}, Total VRAM: {total} MB, Free VRAM: {free} MB")

if total < 15109:  # 15109MB is equivalent to 15GB
    # Display an error message in red text
    print("\033[91mError: Not enough VRAM available. Please change the runtime to a GPU with at least 15GB VRAM.\033[0m")
    raise SystemExit
else:
    print("\033[92mYou have enough VRAM to continue\033[0m")


### Get CUDA version

In [ ]:
!nvcc -V

In [ ]:
%pwd

### Install dependencies

In [ ]:
#Install autotrain
%pip install -U autotrain-advanced

In [ ]:
# setup will install some dependencies
!autotrain setup

### Auotrain help

In [ ]:
!autotrain dreambooth --help

### Toy example

Now let's get our dataset. For this example we will use some dog images: https://huggingface.co/datasets/diffusers/dog-example.

Let's first download it locally:

In [ ]:
%pwd

In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir, repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
%pwd

In [ ]:
%ls

### Set environment variables

In [ ]:
import os

def set_env_variables(project_name = 'test', image_path = 'training_images'):

    # Stable diffusion model
    os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-xl-base-1.0"
    #os.environ["INSTANCE_DIR"] = "dog"
    # The model name
    os.environ["PROJECT_NAME"] = project_name
    #os.environ["VAE_PATH"] = "madebyollin/sdxl-vae-fp16-fix"
    # Training image's dir
    os.environ["IMAGE_PATH"]

# Test the function by calling it
set_env_variables('test-0')

# You can now access these environment variables like this:
model_name = os.environ["MODEL_NAME"]
project_name = os.environ["PROJECT_NAME"]
image_path = os.environ["IMAGE_PATH"]

print(f"MODEL_NAME: {model_name}")
print(f"PROJECT_NAME: {project_name}")
print(f"IMAGE_PATH: {image_path}")


### Run dreambooth training

In [ ]:
!autotrain dreambooth \
  --model $MODEL_NAME \
  --image-path $IMAGE_PATH \
  --project-name $PROJECT_NAME \
  --prompt "photo of sks man" \
  --resolution 1024 \
  --batch-size 1 \
  --num-steps 500 \
  --fp16 \
  --gradient-accumulation 4 \
  --lr 1e-4
# --push-to-hub
# --hub-model-id username/sdxl-dreambooth-model


In [ ]:
! sudo rm -r output_refined

### Inference

In [ ]:
import os

# dir for generated images
OUTPUT_DIR = 'output_images'
# dir for refined images
OUTPUT_DIR_REFINED = OUTPUT_DIR+'_refined'
os.makedirs(OUTPUT_DIR,exist_ok = True)
os.makedirs(OUTPUT_DIR_REFINED,exist_ok = True)

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
)
pipe.to("cuda")
project_name  = os.getenv('PROJECT_NAME')
model_name = os.getenv('MODEL_NAME')
pipe.load_lora_weights(project_name,
                       weight_name="pytorch_lora_weights.safetensors")

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
)
refiner.to("cuda")


prompt = "a photo of sks dog, pixar, cartoon, 3d, headshots, fantasy, 4k, uhd"

for seed in range(10):
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt,
                 generator=generator,
                 num_inference_steps=25)
    image = image.images[0]
    image.save(f"{OUTPUT_DIR}/{seed}.png")
    image = refiner(prompt=prompt, generator=generator, image=image)
    image = image.images[0]
    image.save(f"{OUTPUT_DIR_REFINED}/{seed}.png")



### Zip output dirs
Easily download the generated images.


In [ ]:
import zipfile

import zipfile
import os

def zip_folder(folder_name):
    try:
        # Generate the output zip filename by appending '.zip' to the folder name
        output_zip_filename = f"{folder_name}.zip"

        with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(folder_name):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, folder_name)
                    zipf.write(file_path, arcname)
        print(f'Folder "{folder_name}" has been compressed to "{output_zip_filename}" successfully.')
    except Exception as e:
        print(f'Error: {e}')


In [ ]:
#Compress dirs to download
folder_name = 'output_images'
zip_folder(OUTPUT_DIR)

In [ ]:
zip_folder(OUTPUT_DIR_REFINED)